In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datatable as dt
import tensorflow as tf
from tensorflow import keras
from warnings import filterwarnings
filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

## <span style="background:#818181;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Introduction</span>

<div style="font-size: 1em; font-family: Verdana">
    <b>Hi,</b><br><br>
    I just wanted to share my baseline-model with you guys.<br>
    I've just recently started getting into 'deep learning' and read a lot of basics.<br>
    This is the reason why I decided to use this month competition to get some practice with Neural-Networks.<br><br>
    Also make sure to check out my EDA for TPS November 2021 <a href="https://www.kaggle.com/mlanhenke/tps-11-simple-basic-eda">here</a>. <br><br>
    <em>If you like this notebook or copy any parts of it please make sure to leave an upvote...</em><br><br>
    <em><b>Thanks for taking some time to stop by and read my notebook!</b></em>
</div>



## <span style="background:#818181;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Import Data & Pre-Processing</span>

In [ ]:
def create_row_features(df):
    df['row_sum'] = df.sum(axis=1)
    df['row_mean'] = df.mean(axis=1)
    df['row_std'] = df.std(axis=1)
    return df

In [ ]:
from sklearn.preprocessing import StandardScaler

### load dataframes
df_train = dt.fread('../input/tabular-playground-series-nov-2021/train.csv').to_pandas()
df_test = dt.fread('../input/tabular-playground-series-nov-2021/test.csv').to_pandas()

sample_submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

### split into X, y
X = df_train.drop(columns=['id','target']).copy()
y = df_train['target'].copy()

X_test = df_test.drop(columns='id').copy()

### get row features
X = create_row_features(X)
X_test = create_row_features(X_test)

### standardize data
scaler = StandardScaler()

X = pd.DataFrame(columns=X.columns, data=scaler.fit_transform(X))
X_test = pd.DataFrame(columns=X_test.columns, data=scaler.transform(X_test))

## <span style="background:#818181;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Modeling</span>

In [ ]:
### check gpu before training
print("Tensorflow Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

### define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=20, 
    verbose=0,
    mode='min', 
    baseline=None, 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,
    patience=5,
    mode='min'
)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, InputLayer, Dropout

### create baseline-model
def get_model(name:str):
    model = Sequential(name=name)

    model.add(InputLayer(input_shape=(X.shape[1])))
    model.add(Flatten())

    model.add(Dense(300, activation='relu'))
    model.add(Dropout(rate=0.3))
    
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(rate=0.3))
    
    model.add(Dense(100, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
from sklearn.model_selection import StratifiedKFold

### cross-validation 
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

scores = {fold:None for fold in range(cv.n_splits)}
predictions = []

for fold, (idx_train, idx_valid) in enumerate(cv.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model = get_model(name='Baseline')
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['AUC']
    )

    print('**'*20)
    print(f"Fold {fold+1} || Training")
    print('**'*20)
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        batch_size=1024,
        epochs=1500,
        verbose=0,
        callbacks=[
            early_stopping,
            reduce_lr
        ]
    )
    
    scores[fold] = (history.history)
    
    print(f"Fold {fold+1} || Mean Validation AUC: {np.mean(scores[fold]['val_auc'])}")
    
    prediction = model.predict(X_test).reshape(1,-1)[0]
    predictions.append(prediction)

## <span style="background:#818181;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Evaluation</span>

In [ ]:
### plot train and valid loss over number of epochs
fig, ax = plt.subplots(1, 5, tight_layout=True, figsize=(20,2.5))
ax = ax.flatten()

for fold in range(cv.n_splits):
    df_eval = pd.DataFrame({'train_loss': scores[fold]['loss'], 'valid_loss': scores[fold]['val_loss']})

    mean_train = np.round(np.mean(df_eval['train_loss']),5)
    mean_valid = np.round(np.mean(df_eval['valid_loss']),5)
    delta = np.round(mean_valid - mean_train,5)
    
    sns.lineplot(
        x=df_eval.index,
        y=df_eval['train_loss'],
        label='train_loss',
        ax = ax[fold]
    )

    sns.lineplot(
        x=df_eval.index,
        y=df_eval['valid_loss'],
        label='valid_loss',
        ax = ax[fold]
    )
    
    ax[fold].set_ylabel('')
    ax[fold].set_xlabel(f"Fold {fold+1}\nmean_train: {mean_train}\nmean_valid: {mean_valid}\ndelta: {delta}", fontstyle='italic')

sns.despine()

## <span style="background:#818181;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Submission</span>

In [ ]:
### average predictions over each fold and create submission file
sample_submission['target'] = np.mean(np.column_stack(predictions), axis=1)
sample_submission.to_csv('./nn_baseline.csv', index=False)